In [23]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
torch.cuda.empty_cache()

In [25]:
path_img_106 = []
path_img_ngt = []

In [26]:
root = r"C:\Users\lucyc\Desktop\DatSet_Activate\105_face"
dir_list = os.listdir(root)
for dir_person in dir_list:
    path_img_106.append([root+"\\"+dir_person+"\\"+x for x in os.listdir(root + "\\" + dir_person) if x[-1] == "g" or x[-1] == "G"])

root = r"C:\Users\lucyc\Desktop\DatSet_Activate\diff_face"
path_img_ngt = [root+"\\"+x for x in os.listdir(root)[:50000]]

In [27]:
path_data_trp = []

#count = 0
# for cla_imgs in path_img_106:
#     for i in range(1, len(cla_imgs), 2):
#         path_data_trp.append((cla_imgs[i], cla_imgs[i-1], path_img_ngt[count]))
#         count += 1

# for cla_i in range(1, len(path_img_106), 2):
#     j = 1
#     while j < len(path_img_106[cla_i-1]) and j//2 < len(path_img_106[cla_i]):
#         path_data_trp.append((path_img_106[cla_i-1][j-1], path_img_106[cla_i-1][j], path_img_106[cla_i][j//2]))
#         j += 2
#         count += 1

# for cla_i in range(2, len(path_img_106), 2):
#     j = 1
#     while j < len(path_img_106[cla_i-1]) and j//2 < len(path_img_106[cla_i]):
#         path_data_trp.append((path_img_106[cla_i-1][j-1], path_img_106[cla_i-1][j], path_img_106[cla_i][j//2]))
#         j += 2
#         count += 1


import random

def random_pop(lst):
    if lst:  # 检查列表是否非空
        index = random.randrange(len(lst))  # 选择一个随机索引
        return lst.pop(index)  # 弹出并返回该索引对应的元素
    else:
        return None  # 如果列表为空，则返回None

def get_ngt(i):
    all_path = []
    
    def rebuild():
        nonlocal all_path
        for j, x in enumerate(path_img_106):
            if j == i:
                continue
            all_path += x
            
    rebuild()
    while True:
        select = random_pop(all_path)
        if select != None:
            yield select
        else:
            rebuild()
            yield random_pop(all_path)

for i in range(len(path_img_106)):
    getngt = get_ngt(i)
    for j in range(len(path_img_106[i])):
        for k in range(j+1,len(path_img_106[i])):
            path_data_trp.append([path_img_106[i][j], path_img_106[i][k], next(getngt)])


In [28]:
len(path_data_trp)

1515807

In [29]:
test_path_data_trp = []
valid_path_data_trp = []
train_path_data_trp = []

for g in path_data_trp:
    if "3" in g[0] or "3" in g[1] or "3" in g[2]:
        train_path_data_trp.append(g)
    elif "4" in g[0] or "4" in g[1] or "4" in g[2]:
        valid_path_data_trp.append(g)
    else:
        test_path_data_trp.append(g)

In [30]:
import random
test_path_data_trp = random.sample(test_path_data_trp, len(test_path_data_trp))
valid_path_data_trp = random.sample(valid_path_data_trp, len(valid_path_data_trp)//50)
train_path_data_trp = random.sample(train_path_data_trp, len(train_path_data_trp)//100)

In [31]:
print(len(test_path_data_trp),len(valid_path_data_trp),len(train_path_data_trp))

34891 3936 12840


In [32]:
img = Image.open(path_data_trp[0][0])

In [33]:

# 定义转换操作
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # 将PIL图像或NumPy ndarray转换为FloatTensor。
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # 标准化，使用ImageNet的均值和标准差
                         std=[0.229, 0.224, 0.225])
])

from boring import TrpDataSet

# 创建数据集
test_dataset = TrpDataSet(test_path_data_trp, transform=transform)
val_dataset = TrpDataSet(valid_path_data_trp, transform=transform)
train_dataset = TrpDataSet(train_path_data_trp, transform=transform)
# dataset = TrpDataSet(path_data_trp, transform=transform)
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)+1])
# train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset)*0.9), int(len(train_dataset)*0.1)])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=3, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=3, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=3, pin_memory=True)


In [34]:
a = next(iter(train_loader))

In [35]:
a[0].shape

torch.Size([128, 3, 224, 224])

In [36]:
from torchvision import models
# 加载预训练的VGG-16模型
vgg16 = models.vgg16(pretrained=True)

In [37]:
vgg16.classifier[0].in_features

25088

In [39]:
for p in vgg16.parameters():
    p.requires_grad = False

In [40]:
for i in range(18,30):
    print(vgg16.features[i])
    #vgg16.features[i].parameter.requires_grad = True
    for param in vgg16.features[i].parameters():
        param.requires_grad = True
        print("1")

ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1
1
ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1
1
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1
1
ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1
1
ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1
1
ReLU(inplace=True)


In [41]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [42]:
class SiameseCNN(nn.Module):

    def __init__(self):
        super(SiameseCNN, self).__init__()
        
        self.last_conv_filt = 256
        self.dens_size_sub = 10
        self.dens_size = self.dens_size_sub*self.dens_size_sub*self.last_conv_filt
        
        # 定义网络层
        self.conv1 = nn.Conv2d(3, 16, 3)  #222 -> 111
        self.conv2 = nn.Conv2d(16, 32, 3) #109 -> 54
        self.conv3 = nn.Conv2d(32, 64, 3) #52 -> 26
        self.conv4 = nn.Conv2d(64, 128, 3) #24 -> 12
        self.conv5 = nn.Conv2d(128, self.last_conv_filt, 3) #10
        # 定义全连接层
        self.dens = nn.Linear(self.dens_size, 128)

    def forward(self, x):
        # 定义前向传播
        x = F.relu(self.conv1(x)) #222
        x = F.max_pool2d(x, 2) #111
        x = F.relu(self.conv2(x)) #109
        x = F.max_pool2d(x, 2) #54
        x = F.relu(self.conv3(x)) #52
        x = F.max_pool2d(x, 2) #26
        x = F.relu(self.conv4(x)) #24
        x = F.max_pool2d(x, 2) #12
        x = F.relu(self.conv5(x)) #10
        x = x.flatten(start_dim=1)
        x = F.relu(self.dens(x))
        return x


In [43]:
class SiameseCNN_VGG16(nn.Module):

    def __init__(self):
        super(SiameseCNN_VGG16, self).__init__()
        # 定义网络层
        self.vgg16 = model = nn.Sequential(
                            vgg16.features,
                            vgg16.avgpool
        )
        self.feature = vgg16.classifier[0].in_features
        # 定义全连接层
        # self.dens = nn.Linear(self.feature, 1024)
        # self.drop = nn.Dropout(p=0.5, inplace=False)
        # self.dens2 = nn.Linear(1024, 1024)

    def forward(self, x):
        # 定义前向传播
        #x = x.flatten(start_dim=1)
        x = self.vgg16(x)
        x = torch.flatten(x, 1)
        # x = F.relu(self.dens(x))
        # x = self.drop(x)
        # x = F.relu(self.dens2(x))
        return x


In [44]:
class TripLoss(nn.Module):
    def __init__(self):
        super(TripLoss, self).__init__()
        # 这里可以初始化任何需要的参数或层

    def forward(self, anchor, positive, negative):
        # 自定义损失的计算逻辑
        # 例如，我们可以计算平均绝对误差     
        # 计算锚点与正负样本之间的距离
        distance_positive = (anchor - positive).pow(2).sum(1)  # 对每个元素平方后求和
        distance_negative = (anchor - negative).pow(2).sum(1)
        
        # 计算三元组损失
        losses = F.relu(distance_positive - distance_negative + 3)
        
        # 返回平均损失
        return losses.mean()

In [45]:
torch.cuda.empty_cache()
torch.autograd.set_detect_anomaly(True)
# 构建模型
#model = SiameseCNN()
model = SiameseCNN_VGG16()
print(model)

model.to(device)  # 将模型发送到GPU，如果有的话

# 定义损失函数和优化器
criterion = TripLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练模型
num_epochs = 30

for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式

    loss_acc = 0.
    train_num = 0
    for inputs in train_loader:
        anchor = inputs[0].to(device)
        positive = inputs[1].to(device)
        negative = inputs[2].to(device)
        
        # 前向传播
        anchor = model(anchor)
        positive = model(positive)
        negative = model(negative)

        optimizer.zero_grad()  # 清除之前的梯度
        loss = criterion(anchor, positive, negative)
        # 反向传播和优化
        loss.backward()  # 反向传播计算当前的梯度
        optimizer.step()  # 更新参数

        loss_acc += loss.item()
        train_num += 1

        #print(loss)

    model.eval() 
    val_loss_acc = 0
    val_num = 0
    with torch.no_grad():
        for inputs in val_loader:
            anchor = inputs[0].to(device)
            positive = inputs[1].to(device)
            negative = inputs[2].to(device)
            
            anchor = model(anchor)
            positive = model(positive)
            negative = model(negative)

            loss = criterion(anchor, positive, negative)
            #loss += abs(5-criterion(anchor, negative))
            val_loss_acc += loss
            val_num += 1

    print("Epoch [{}/{}], Loss: {:.2f}, Val_loss: {:.2f}".format(epoch+1, num_epochs, loss_acc/train_num, val_loss_acc/val_num))


SiameseCNN_VGG16(
  (vgg16): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
   

KeyboardInterrupt: 

In [46]:
def loss_cal(v1, v2):
    apd = torch.sum((v1 - v2) ** 2, dim=1)
    return apd.mean() # 平均欧几里得距离

model.eval()
tl = iter(test_loader)
pass_num = 0
fail_num = 0
with torch.no_grad():
    for _ in range(250):
        anchor, positive, negative = next(tl)
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        a = model(anchor)
        p = model(positive)
        n = model(negative)
        
        if loss_cal(a,p) < loss_cal(a,n):
            pass_num += 1
        else:
            fail_num += 1
            
        print("ap: {:.2f}, an: {:.2f}".format(loss_cal(a,p),loss_cal(a,n)))

print(pass_num)
    

ap: 15.09, an: 17.17
ap: 15.72, an: 18.54
ap: 11.44, an: 13.71
ap: 18.71, an: 19.48
ap: 13.59, an: 12.78
ap: 14.27, an: 33.61
ap: 21.10, an: 14.08
ap: 14.05, an: 17.78
ap: 13.60, an: 14.35
ap: 14.66, an: 33.29
ap: 18.39, an: 26.68
ap: 10.76, an: 11.51
ap: 22.92, an: 41.89
ap: 14.90, an: 35.70
ap: 18.19, an: 23.12
ap: 12.35, an: 18.88
ap: 14.61, an: 10.00
ap: 16.61, an: 30.87
ap: 11.70, an: 15.12
ap: 13.23, an: 16.20
ap: 11.60, an: 11.13
ap: 9.49, an: 14.11
ap: 20.66, an: 19.79
ap: 16.58, an: 32.67
ap: 16.40, an: 28.39
ap: 15.24, an: 17.84
ap: 19.44, an: 21.71
ap: 16.95, an: 27.66
ap: 15.96, an: 24.82
ap: 16.21, an: 26.78
ap: 12.69, an: 9.67
ap: 15.59, an: 27.69
ap: 15.97, an: 16.59
ap: 16.03, an: 12.51
ap: 16.29, an: 12.24
ap: 22.12, an: 26.07
ap: 16.78, an: 23.91
ap: 18.07, an: 16.85
ap: 18.17, an: 23.22
ap: 17.08, an: 43.02
ap: 16.84, an: 21.75
ap: 16.53, an: 27.05
ap: 16.60, an: 20.33
ap: 15.50, an: 17.74
ap: 14.21, an: 18.84
ap: 20.44, an: 14.85
ap: 11.23, an: 29.39
ap: 19.90, an: 

In [79]:
for a, b in model.named_parameters():
    print(a)
    b.requires_grad = True

vgg16.0.0.weight
vgg16.0.0.bias
vgg16.0.2.weight
vgg16.0.2.bias
vgg16.0.5.weight
vgg16.0.5.bias
vgg16.0.7.weight
vgg16.0.7.bias
vgg16.0.10.weight
vgg16.0.10.bias
vgg16.0.12.weight
vgg16.0.12.bias
vgg16.0.14.weight
vgg16.0.14.bias
vgg16.0.17.weight
vgg16.0.17.bias
vgg16.0.19.weight
vgg16.0.19.bias
vgg16.0.21.weight
vgg16.0.21.bias
vgg16.0.24.weight
vgg16.0.24.bias
vgg16.0.26.weight
vgg16.0.26.bias
vgg16.0.28.weight
vgg16.0.28.bias
dens.weight
dens.bias
dens2.weight
dens2.bias


In [44]:
for a, b in model.named_parameters():
    print(a)
    print(b.requires_grad)

conv1.weight
True
conv1.bias
True
conv2.weight
True
conv2.bias
True
conv3.weight
True
conv3.bias
True
conv4.weight
True
conv4.bias
True
conv5.weight
True
conv5.bias
True
dens.weight
True
dens.bias
True


In [25]:
torch.save(model, 'face_v2_full.pth')

In [26]:
torch.save(model.state_dict(), 'face_v2.pth')